In [2]:
from z3 import *

# Definindo o tamanho do BitVec
n = 8  # Por exemplo, 8 bits

# Definindo as variáveis BitVec
a = BitVec('a', n)
b = BitVec('b', n)
x = BitVec('x', n)
y = BitVec('y', n)
z = BitVec('z', n)

# Estado inicial: x = a, y = b, z = 0
initial_state = And(x == a, y == b, z == 0, a >= 0, b >= 0)

# Relação de transição
next_x = If(y % 2 == 0, x << 1, x)       # Se y é par, x <<= 1
next_y = If(y % 2 == 0, y >> 1, y - 1)   # Se y é par, y >>= 1, senão y -= 1
next_z = If(y % 2 == 0, z, z + x)        # Se y é ímpar, z += x

# Estado de erro
error_state = Or(UGT(next_x, BitVecVal(2**n - 1, n)), UGT(next_z, BitVecVal(2**n - 1, n)))

# Propriedade do invariante a * b = x * y + z
invariant_property = (x * y + z == a * b)

# Função para verificar a propriedade do invariante
def check_invariant():
    s = Solver()
    # Adicionando o estado inicial
    s.add(initial_state)
    # Adicionando a propriedade que queremos manter como invariante
    s.add(Not(invariant_property))
    
    # Verificando se a propriedade do invariante é falsa
    if s.check() == sat:
        print("Invariante não é mantido.")
        print("Modelo que quebra a propriedade:", s.model())
    else:
        print("Invariante é mantido.")

# Função para verificar a segurança do programa
def check_safety():
    s = Solver()
    # Condições adicionais para a segurança
    safety_conditions = And(a < BitVecVal(2**(n//2), n), b < BitVecVal(2**(n//2), n))
    # Adicionando o estado inicial e as condições de segurança
    s.add(initial_state)
    s.add(safety_conditions)
    
    # Verificando se o estado de erro é acessível
    s.add(error_state)
    
    if s.check() == sat:
        print("Estado de erro acessível sob as condições de segurança.")
        print("Modelo que atinge estado de erro:", s.model())
    else:
        print("Estado de erro não é acessível sob as condições de segurança.")

# Verificar a propriedade do invariante
check_invariant()

# Verificar a segurança do programa
check_safety()


Invariante é mantido.
Estado de erro não é acessível sob as condições de segurança.
